# Assignment 4: Final Project
Authors: Naomi Buell and Richie Rivera

## *Instructions:*

1. Choose a dataset: You get to decide which dataset you want to work on. The data set must be different from the ones used in previous homeworks You can work on a problem from your job, or something you are interested in. You may also obtain a dataset from sites such as Kaggle, Data.Gov, Census Bureau, USGS or other open data portals. 

2. Select one of the methodologies studied in weeks 1-10, and another methodology from weeks 11-15 to apply in the new dataset selected.

3. To complete this task:
    - Describe the problem you are trying to solve.
    - Describe your datasest and what you did to prepare the data for analysis. 
    - Methodologies you used for analyzing the data
    - What's the purpose of the analysis performed
    - Make your conclusions from your analysis. Please be sure to address the business impact (it could be of any domain) of your solution.

## Introduction

### The Problem: 

This project seeks to answer the research question: **What factors influence user reviews for Steam video games?** Steam is a popular digital distribution platform for video games, and user reviews play a significant role in shaping the perception and success of games on the platform. Understanding the factors that influence these reviews can provide valuable insights for game developers, marketers, and the gaming community.

Importing Libraries.

In [1]:
# Load libraries
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score, recall_score, classification_report
from imblearn.under_sampling import RandomUnderSampler
from sklearn.tree import plot_tree
from sklearn.svm import SVC
from sklearn.preprocessing import RobustScaler
from collections import Counter
import numpy as np
import zipfile
import io
import requests
import seaborn as sns

import requests
import json
from datetime import datetime, timedelta
import time
from concurrent.futures import ThreadPoolExecutor
import os
import json
import pickle
import zipfile
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import re
from nltk.corpus import stopwords
import shutil

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nbuell/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Data Selection

> Describe your datasest and what you did to prepare the data for analysis. 

For our project, we'll analyze Steam game reviews for some of the top 100 best selling games to explore how factors such as game genre, release date, and user demographics impact reviews. We'll use web scraping techniques to collect review data from Steam's platform.

## Import Data

In [2]:
# Copy in from 620 final

## Exploratory Data Analysis

In [3]:
# Copy in from 620 final

We also perform label encoding on other categorical variables before we feed data to tree-based machine learning algorithms.

In [44]:
# One-hot encode nominal categorical variables
df_dummies = df_fe.copy()
df_dummies = pd.get_dummies(df_dummies, columns=['job', 'marital', 'housing', 'loan', 'contact', 'poutcome', 'y'], drop_first=True)
df_dummies = df_dummies.rename(columns={'y_yes': 'y'})

df_dummies.head()

,age,education,month,day_of_week,campaign,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,...,job_technician,job_unemployed,marital_married,marital_single,housing_yes,loan_yes,contact_telephone,poutcome_nonexistent,poutcome_success,y
0,56,4,5,2,1,0,1.1,93.994,-36.4,4.857,...,False,False,True,False,False,False,True,True,False,False
1,57,12,5,2,1,0,1.1,93.994,-36.4,4.857,...,False,False,True,False,False,False,True,True,False,False
2,37,12,5,2,1,0,1.1,93.994,-36.4,4.857,...,False,False,True,False,True,False,True,True,False,False
3,40,6,5,2,1,0,1.1,93.994,-36.4,4.857,...,False,False,True,False,False,False,True,True,False,False
4,56,12,5,2,1,0,1.1,93.994,-36.4,4.857,...,False,False,True,False,False,True,True,True,False,False


#### Standardize Features

Finally, we standardize the features to have mean 0 and standard deviation 1. Although the tree-based models from the previous assignment do not require standardization, this is important for SVMs since they are sensitive to the scale of the input features. We are using `RobustScaler()` to reduce the influence of outliers.

In [45]:
# Identify numerical columns
num_cols = df_dummies.select_dtypes(include=[np.number]).columns
df_standardized = df_dummies.copy()
scaler = RobustScaler()

# Fit and transform the numerical columns
df_standardized[num_cols] = scaler.fit_transform(df_standardized[num_cols])

df_standardized.head()

,age,education,month,day_of_week,campaign,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,...,job_technician,job_unemployed,marital_married,marital_single,housing_yes,loan_yes,contact_telephone,poutcome_nonexistent,poutcome_success,y
0,1.200000,-1.142857,-0.333333,-1.0,-0.5,0.0,0.0,0.598477,0.857143,0.0,...,False,False,True,False,False,False,True,True,False,False
1,1.266667,0.000000,-0.333333,-1.0,-0.5,0.0,0.0,0.598477,0.857143,0.0,...,False,False,True,False,False,False,True,True,False,False
2,-0.066667,0.000000,-0.333333,-1.0,-0.5,0.0,0.0,0.598477,0.857143,0.0,...,False,False,True,False,True,False,True,True,False,False
3,0.133333,-0.857143,-0.333333,-1.0,-0.5,0.0,0.0,0.598477,0.857143,0.0,...,False,False,True,False,False,False,True,True,False,False
4,1.200000,0.000000,-0.333333,-1.0,-0.5,0.0,0.0,0.598477,0.857143,0.0,...,False,False,True,False,False,True,True,True,False,False


Our data is ready for us train our models.

## Create Model 1 (from weeks 1-10)

Firstly, we anticipate the need to compare our models so we'll compile all of our models' results into one list.

In [46]:
all_model_results = []

In this section, we train and validate the SVM model and tune hyper-parameters. For all models, we split the data into 70% for training and 30% for testing. We will start with a linear kernel and the regularization parameter C=1.0 (default).

In [47]:
# Separate features and target
X = df_standardized.drop(columns=['y'])
y = df_standardized['y']

# Randomly split the X and y arrays into 30 percent test data and 70 percent training data.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1, stratify=y
)

# Create and train the SVM model
svm_linear = SVC(kernel='linear', C=1.0, random_state=1)
svm_linear.fit(X_train, y_train)

# Display a detailed classification report
y_pred_svm = svm_linear.predict(X_test) # Predictions on test set
print("\nClassification Report on Test Set:")
print(classification_report(y_test, y_pred_svm))

# Accuracy on balanced training set
y_train_pred = svm_linear.predict(X_train) # Predictions on training set
train_acc = accuracy_score(y_train, y_train_pred)
print('Training Accuracy: %.2f%%' % (train_acc * 100))

# Accuracy on test set
test_acc = accuracy_score(y_test, y_pred_svm)
print('Test Accuracy: %.2f%%' % (test_acc * 100))

# Recall on the test set
test_recall_svm = recall_score(y_test, y_pred_svm)
print('Final Model Recall on Test Set:  %.2f%%' % (test_recall_svm * 100))


Classification Report on Test Set:
              precision    recall  f1-score   support

       False       0.91      0.99      0.94     10197
        True       0.63      0.19      0.30      1277

    accuracy                           0.90     11474
   macro avg       0.77      0.59      0.62     11474
weighted avg       0.88      0.90      0.87     11474

Training Accuracy: 89.83%
Test Accuracy: 89.78%
Final Model Recall on Test Set:  19.26%


In [48]:
report = classification_report(y_test, y_pred_svm, output_dict=True)

# Adding an entry to the model results list
all_model_results.append({
    'model': 'SVM (Linear)',
    'train_acc': accuracy_score(y_train, y_train_pred),
    'test_acc': report['accuracy'],
    'test_recall': recall_score(y_test, y_pred_svm),
    'precision_macro': report['macro avg']['precision'],
    'recall_macro': report['macro avg']['recall'],
    'f1_macro': report['macro avg']['f1-score'],
    'precision_weighted': report['weighted avg']['precision'],
    'recall_weighted': report['weighted avg']['recall'],
    'f1_weighted': report['weighted avg']['f1-score']
})

The SVM model predicts bank subscriptions with 89.78% accuracy on the test set. Training and test accuracies are similar and do not indicate overfitting. The F1 score for the positive class (subscribed) is 0.30.

We will also explore using the RBF kernel (default).

In [49]:
# Create and train the SVM model
svm_rbf = SVC(kernel='rbf', C=1.0, random_state=1)
svm_rbf.fit(X_train, y_train)

# Display a more detailed classification report
y_train_pred = svm_rbf.predict(X_train) # Predictions on training set
y_pred_svm = svm_rbf.predict(X_test) # Predictions on test set
print("\nClassification Report on Test Set:")
print(classification_report(y_test, y_pred_svm))

# Accuracy on training set
train_acc = accuracy_score(y_train, y_train_pred)
print('Training Accuracy: %.2f%%' % (train_acc * 100))

# Accuracy on test set
test_acc = accuracy_score(y_test, y_pred_svm)
print('Test Accuracy: %.2f%%' % (test_acc * 100))

# Calculate final recall on the test set
test_recall_svm = recall_score(y_test, y_pred_svm)
print('Final Model Recall on Test Set: %.2f%%' % (test_recall_svm * 100))


Classification Report on Test Set:
              precision    recall  f1-score   support

       False       0.91      0.98      0.94     10197
        True       0.63      0.21      0.32      1277

    accuracy                           0.90     11474
   macro avg       0.77      0.60      0.63     11474
weighted avg       0.88      0.90      0.88     11474

Training Accuracy: 90.51%
Test Accuracy: 89.82%
Final Model Recall on Test Set: 21.30%


In [50]:
# Get the report as a dictionary (uses y_pred_svm from the rbf model)
report = classification_report(y_test, y_pred_svm, output_dict=True)

# Adding an entry to the model results list
all_model_results.append({
    'model': 'SVM (rbf)',
    'train_acc': accuracy_score(y_train, y_train_pred),
    'test_acc': report['accuracy'],
    'test_recall': recall_score(y_test, y_pred_svm),
    'precision_macro': report['macro avg']['precision'],
    'recall_macro': report['macro avg']['recall'],
    'f1_macro': report['macro avg']['f1-score'],
    'precision_weighted': report['weighted avg']['precision'],
    'recall_weighted': report['weighted avg']['recall'],
    'f1_weighted': report['weighted avg']['f1-score']
})

The test accuracy is slightly improved with the RBF kernel (from 89.78% to 89.82%) and the F1 score is also improved (from 0.30 to 0.32). 

Given these improved results, we will proceed with hyper-parameter tuning to see if we can improve performance further.

In [51]:
# Tune hyper-parameters 
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.01, 0.001],
}
grid_search_svm = GridSearchCV(
    estimator=svm_rbf,
    param_grid=param_grid,
    scoring='recall',
    cv=5,
    n_jobs=-1
)

# Train the model
grid_search_svm.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(random_state=1), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10],
                         'gamma': ['scale', 'auto', 0.01, 0.001]},
             scoring='recall')

In [52]:
# Display best parameters
print("Best Hyper-parameters:", grid_search_svm.best_params_)

# Display a more detailed classification report
y_pred_svm = grid_search_svm.predict(X_test) # Predictions on test set
print("\nClassification Report on Test Set:")
print(classification_report(y_test, y_pred_svm))

# Accuracy on training set
y_train_pred = grid_search_svm.predict(X_train) # Predictions on training set
train_acc = accuracy_score(y_train, y_train_pred)
print('Training Accuracy: %.2f%%' % (train_acc * 100))

# Accuracy on test set
test_acc = accuracy_score(y_test, y_pred_svm)
print('Test Accuracy: %.2f%%' % (test_acc * 100))

# Calculate final recall on the test set
test_recall_svm = recall_score(y_test, y_pred_svm)
print('Final Model Recall on Test Set:  %.2f%%' % (test_recall_svm * 100))
print('Best Cross-Validation Recall Score:  %.2f%%' % (grid_search_svm.best_score_ * 100))

Best Hyper-parameters: {'C': 10, 'gamma': 'scale'}

Classification Report on Test Set:
              precision    recall  f1-score   support

       False       0.91      0.98      0.94     10197
        True       0.60      0.24      0.34      1277

    accuracy                           0.90     11474
   macro avg       0.75      0.61      0.64     11474
weighted avg       0.88      0.90      0.88     11474

Training Accuracy: 91.76%
Test Accuracy: 89.73%
Final Model Recall on Test Set:  24.12%
Best Cross-Validation Recall Score:  24.52%


In [53]:
# Get the report as a dictionary (uses y_pred_svm from grid search)
report = classification_report(y_test, y_pred_svm, output_dict=True)

# Adding an entry to the model results list
all_model_results.append({
    'model': 'SVM (GridSearch)',
    'train_acc': accuracy_score(y_train, y_train_pred),
    'test_acc': report['accuracy'],
    'test_recall': recall_score(y_test, y_pred_svm),
    'precision_macro': report['macro avg']['precision'],
    'recall_macro': report['macro avg']['recall'],
    'f1_macro': report['macro avg']['f1-score'],
    'precision_weighted': report['weighted avg']['precision'],
    'recall_weighted': report['weighted avg']['recall'],
    'f1_weighted': report['weighted avg']['f1-score'],
    'best_cv_score': grid_search_svm.best_score_,
    'best_params': grid_search_svm.best_params_
})

The hyper-parameter tuning yielded an accuracy of 89.73%, which is a slight decrease from the previous RBF model. However, since the grid search was optimized for recall, we see an improvement in recall from 21.3% to 24.12%, meaning the model can identify more of the positive class (subscribed) instances. The F1 score also improved slightly from 0.32 to 0.34.

Lastly, we try balancing the training data.

In [54]:
# Balance training data
print(f"Unbalanced training set size: {X_train.shape[0]:,}")
print(f"Unbalanced training set class distribution: {Counter(y_train)}")
undersampler = RandomUnderSampler(random_state=1)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)
print(f"Resampled training set size: {X_train_resampled.shape[0]:,}")
print(f"Resampled training set class distribution: {Counter(y_train_resampled)}")

# Train the optimized model on the balanced data
grid_search_svm.fit(X_train_resampled, y_train_resampled)

# Display a more detailed classification report
y_pred_svm = grid_search_svm.predict(X_test) # Predictions on test set
print("\nClassification Report on Test Set:")
print(classification_report(y_test, y_pred_svm))

# Accuracy on training set
y_train_pred = grid_search_svm.predict(X_train_resampled) # Predictions on (balanced) training set
train_acc = accuracy_score(y_train_resampled, y_train_pred)
print('Training Accuracy: %.2f%%' % (train_acc * 100))
print('Best Cross-Validation Recall Score on Training Set: %.2f%%' % (grid_search_svm.best_score_ * 100))

# Accuracy on test set
test_acc = accuracy_score(y_test, y_pred_svm)
print('Test Accuracy: %.2f%%' % (test_acc * 100))

# Calculate final recall on the test set
test_recall_svm = recall_score(y_test, y_pred_svm)
print('Final Model Recall on Test Set:  %.2f%%' % (test_recall_svm * 100))

Unbalanced training set size: 26,771
Unbalanced training set class distribution: Counter({False: 23790, True: 2981})
Resampled training set size: 5,962
Resampled training set class distribution: Counter({False: 2981, True: 2981})

Classification Report on Test Set:
              precision    recall  f1-score   support

       False       0.95      0.72      0.82     10197
        True       0.24      0.70      0.36      1277

    accuracy                           0.72     11474
   macro avg       0.60      0.71      0.59     11474
weighted avg       0.87      0.72      0.77     11474

Training Accuracy: 71.15%
Best Cross-Validation Recall Score on Training Set: 70.41%
Test Accuracy: 72.01%
Final Model Recall on Test Set:  70.48%


In [55]:
# Get the report as a dictionary
report = classification_report(y_test, y_pred_svm, output_dict=True)

# Adding an entry to the model results list
all_model_results.append({
    'model': 'SVM (GridSearch, Undersampled)',
    'train_acc': accuracy_score(y_train_resampled, y_train_pred),
    'test_acc': report['accuracy'],
    'test_recall': recall_score(y_test, y_pred_svm),
    'precision_macro': report['macro avg']['precision'],
    'recall_macro': report['macro avg']['recall'],
    'f1_macro': report['macro avg']['f1-score'],
    'precision_weighted': report['weighted avg']['precision'],
    'recall_weighted': report['weighted avg']['recall'],
    'f1_weighted': report['weighted avg']['f1-score'],
    'best_cv_score': grid_search_svm.best_score_,
    'best_params': grid_search_svm.best_params_
})

Balancing the data made the accuracy drop significantly (nearly 20 percentage points), but we do see a substantial improvement in recall (from 24.12% to 70.48%). This means the model is able to identify a much larger proportion of the positive class (subscribed) instances, but at the cost of many more false positives. I.e., many more clients are incorrectly predicted to subscribe when they do not, but we are able to identify many more clients who do subscribe. If the bank has the resources to cast a wider net with more potential clients, this trade-off may be acceptable.

## Create Model 2 (from weeks 11-15)

We compare the results of the SVM model above with the decision tree, random forest, and AdaBoost models from the previous homework. We detail the model parameters and results of evaluation metrics for each experiment below and compare them with the SVM result.

#### Decision Tree 1

Decision tree with max depth of 4.

In [56]:
# Create and fit decision tree 1
tree_model_1 = DecisionTreeClassifier(criterion='gini',
                                     max_depth=4,
                                     random_state=1)
tree_model_1.fit(X_train, y_train)

# Display a detailed classification report
y_pred_tree1 = tree_model_1.predict(X_test) # Predictions on test set
print("\nClassification Report on Test Set:")
print(classification_report(y_test, y_pred_tree1))

# Accuracy on training set
y_train_pred_tree1 = tree_model_1.predict(X_train) # Predictions on training set
train_acc = accuracy_score(y_train, y_train_pred_tree1)
print('Training Accuracy: %.2f%%' % (train_acc * 100))

# Accuracy on test set
test_acc_tree1 = accuracy_score(y_test, y_pred_tree1)
print('Test Accuracy: %.2f%%' % (test_acc_tree1 * 100))

# Recall on the test set
test_recall_tree1 = recall_score(y_test, y_pred_tree1)
print('Recall on Test Set: %.2f%%' % (test_recall_tree1 * 100))


Classification Report on Test Set:
              precision    recall  f1-score   support

       False       0.91      0.99      0.95     10197
        True       0.69      0.18      0.28      1277

    accuracy                           0.90     11474
   macro avg       0.80      0.58      0.61     11474
weighted avg       0.88      0.90      0.87     11474

Training Accuracy: 90.06%
Test Accuracy: 89.95%
Recall on Test Set: 17.70%


In [57]:
# Get the report as a dictionary
report = classification_report(y_test, y_pred_tree1, output_dict=True)

# Adding an entry to the model results list
all_model_results.append({
    'model': 'Decision Tree (Depth 4)',
    'train_acc': accuracy_score(y_train, y_train_pred_tree1),
    'test_acc': report['accuracy'],
    'test_recall': recall_score(y_test, y_pred_tree1),
    'precision_macro': report['macro avg']['precision'],
    'recall_macro': report['macro avg']['recall'],
    'f1_macro': report['macro avg']['f1-score'],
    'precision_weighted': report['weighted avg']['precision'],
    'recall_weighted': report['weighted avg']['recall'],
    'f1_weighted': report['weighted avg']['f1-score']
})

The decision tree model with a max depth of 4 was 89.95% accurate on the test data, much better than the final SVM model (72.01%), but had lower recall (17.70% vs. 70.48%) and F1 score (0.28 vs. 0.36) for the positive class (subscribed).

#### Decision Tree 2

Decision tree pruned to have a max depth of 2.

In [58]:
# Create and fit a decision tree model
tree_model_2 = DecisionTreeClassifier(criterion='gini',
                                     max_depth=2, # Decrease max depth to 2
                                     random_state=1)
tree_model_2.fit(X_train, y_train)

# Display a detailed classification report
y_pred_tree2 = tree_model_2.predict(X_test) # Predictions on test set
print("\nClassification Report on Test Set:")
print(classification_report(y_test, y_pred_tree2))

# Accuracy on training set
y_train_pred_tree2 = tree_model_2.predict(X_train) # Predictions on training set
train_acc = accuracy_score(y_train, y_train_pred_tree2)
print('Training Accuracy: %.2f%%' % (train_acc * 100))

# Accuracy on test set
test_acc_tree2 = accuracy_score(y_test, y_pred_tree2)
print('Test Accuracy: %.2f%%' % (test_acc_tree2 * 100))

# Recall on the test set
test_recall_tree2 = recall_score(y_test, y_pred_tree2)
print('Recall on Test Set: %.2f%%' % (test_recall_tree2 * 100))



Classification Report on Test Set:
              precision    recall  f1-score   support

       False       0.91      0.99      0.95     10197
        True       0.70      0.17      0.28      1277

    accuracy                           0.90     11474
   macro avg       0.80      0.58      0.61     11474
weighted avg       0.88      0.90      0.87     11474

Training Accuracy: 90.02%
Test Accuracy: 89.96%
Recall on Test Set: 17.31%


In [59]:
# Get the report as a dictionary
report = classification_report(y_test, y_pred_tree2, output_dict=True)

# Adding an entry to the model results list
all_model_results.append({
    'model': 'Decision Tree (Depth 2)',
    'train_acc': accuracy_score(y_train, y_train_pred_tree2),
    'test_acc': report['accuracy'],
    'test_recall': recall_score(y_test, y_pred_tree2),
    'precision_macro': report['macro avg']['precision'],
    'recall_macro': report['macro avg']['recall'],
    'f1_macro': report['macro avg']['f1-score'],
    'precision_weighted': report['weighted avg']['precision'],
    'recall_weighted': report['weighted avg']['recall'],
    'f1_weighted': report['weighted avg']['f1-score']
})

The decision tree model with a max depth of 2 was 89.96% accurate, which is still better than the final SVM model (72.01%). However, it had even lower recall than the previous decision tree (17.31%), making the SVM model preferable.

#### Random Forest 1

Random forest with 25 trees.

In [60]:
# Generate RF model
forest_1 = RandomForestClassifier(n_estimators=25,
                                random_state=1,
                                n_jobs=2)
forest_1.fit(X_train, y_train)

# Display a detailed classification report
y_pred_forest1 = forest_1.predict(X_test) # Predictions on test set
print("\nClassification Report on Test Set:")
print(classification_report(y_test, y_pred_forest1))

# Accuracy on training set
y_train_pred_forest1 = forest_1.predict(X_train) # Predictions on training set
train_acc = accuracy_score(y_train, y_train_pred_forest1)
print('Training Accuracy: %.2f%%' % (train_acc * 100))

# Accuracy on test set
test_acc_forest1 = accuracy_score(y_test, y_pred_forest1)
print('Test Accuracy: %.2f%%' % (test_acc_forest1 * 100))

# Recall on the test set
test_recall_forest1 = recall_score(y_test, y_pred_forest1)
print('Recall on Test Set:  %.2f%%' % (test_recall_forest1 * 100))


Classification Report on Test Set:
              precision    recall  f1-score   support

       False       0.92      0.97      0.94     10197
        True       0.54      0.30      0.38      1277

    accuracy                           0.89     11474
   macro avg       0.73      0.63      0.66     11474
weighted avg       0.87      0.89      0.88     11474

Training Accuracy: 99.15%
Test Accuracy: 89.32%
Recall on Test Set:  29.68%


In [61]:
# Get the report as a dictionary
report = classification_report(y_test, y_pred_forest1, output_dict=True)

# Adding an entry to the model results list
all_model_results.append({
    'model': 'Random Forest (n=25)',
    'train_acc': accuracy_score(y_train, y_train_pred_forest1),
    'test_acc': report['accuracy'],
    'test_recall': recall_score(y_test, y_pred_forest1),
    'precision_macro': report['macro avg']['precision'],
    'recall_macro': report['macro avg']['recall'],
    'f1_macro': report['macro avg']['f1-score'],
    'precision_weighted': report['weighted avg']['precision'],
    'recall_weighted': report['weighted avg']['recall'],
    'f1_weighted': report['weighted avg']['f1-score']
})

This random forest model is less accurate than the previous decision tree models at 89.36% accuracy in correctly predicting a subscriber, but it is still more accurate than the SVM model. However, since the training accuracy is very high at 99.15% (nearly a perfect fit to the training data!), this may indicate overfitting. The recall of the model is very poor. It is only correctly identifying 29.68% of true positives (subscribers) compared to 70.48% for the SVM model.

#### Random Forest 2

Random forest with grid search over number of trees (10-50, in increments of 5) and max depth (2-6 and None, in increments of 1) using 5 fold cross validation.

In [62]:
# Set up our parameters for grid search
param_grid = {
    'n_estimators': [10, 15, 20, 25, 30, 35, 40, 45, 50],
    'max_depth': [2, 3, 4, 5, 6, None]
}

# Initialize our random forest classifier
forest_2 = RandomForestClassifier(random_state=1)

# Set up the grid search with recall as the scoring metric
grid_search = GridSearchCV(
    estimator=forest_2,
    param_grid=param_grid,
    scoring='recall',
    cv=5,
    n_jobs=-1 # This allows us to use all available cores to speed up the training
)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print(f"Best Hyperparameters: {grid_search.best_params_}")

# Get the best model from the grid search
best_forest = grid_search.best_estimator_

Best Hyperparameters: {'max_depth': None, 'n_estimators': 25}


In [63]:
# Display a more detailed classification report
y_pred_best_forest = best_forest.predict(X_test)
print("\nClassification Report on Test Set:")
print(classification_report(y_test, y_pred_best_forest))

# Accuracy on training set
y_train_pred = best_forest.predict(X_train)
train_acc = accuracy_score(y_train, y_train_pred)
print('Training Accuracy: %.2f%%' % (train_acc * 100))

# Accuracy on test set
test_acc = accuracy_score(y_test, y_pred_best_forest)
print('Test Accuracy: %.2f%%' % (test_acc * 100))

# Calculate final recall on the test set
test_recall_rf2 = recall_score(y_test, y_pred_best_forest)
print('Final Model Recall on Test Set:  %.2f%%' % (test_recall_rf2 * 100))
print('Best Cross-Validation Recall Score:  %.2f%%' % (grid_search.best_score_ * 100))


Classification Report on Test Set:
              precision    recall  f1-score   support

       False       0.92      0.97      0.94     10197
        True       0.54      0.30      0.38      1277

    accuracy                           0.89     11474
   macro avg       0.73      0.63      0.66     11474
weighted avg       0.87      0.89      0.88     11474

Training Accuracy: 99.15%
Test Accuracy: 89.32%
Final Model Recall on Test Set:  29.68%
Best Cross-Validation Recall Score:  28.14%


In [64]:
# Get the report as a dictionary
report = classification_report(y_test, y_pred_best_forest, output_dict=True)

# Adding an entry to the model results list
all_model_results.append({
    'model': 'Random Forest (GridSearch)',
    'train_acc': accuracy_score(y_train, y_train_pred),
    'test_acc': report['accuracy'],
    'test_recall': recall_score(y_test, y_pred_best_forest),
    'precision_macro': report['macro avg']['precision'],
    'recall_macro': report['macro avg']['recall'],
    'f1_macro': report['macro avg']['f1-score'],
    'precision_weighted': report['weighted avg']['precision'],
    'recall_weighted': report['weighted avg']['recall'],
    'f1_weighted': report['weighted avg']['f1-score'],
    'best_cv_score': grid_search.best_score_,
    'best_params': grid_search.best_params_
})

This random forest model is even less accurate than the prior random forest, at 89.32% accuracy in correctly predicting subscribers in the test data, but this still performs better than the SVM (70.48%). Unfortunately, since the training accuracy is very high at 98.79% (nearly a perfect fit to the training data!), this one also shows signs of overfitting. Additionally, the recall of the model (29.68%) is very poor compared to the SVM (70.48%).

#### AdaBoost 1

Adaboost with 50 estimators and learning rate of 1.

In [65]:
# train the adaboost algorithm
adaboost_1 = AdaBoostClassifier(
    n_estimators=50,
    learning_rate=1,
    random_state=1
)

# Train our model
adaboost_1.fit(X_train, y_train)

# Display a more detailed classification report
y_pred_ada_1 = adaboost_1.predict(X_test)
print("\nClassification Report on Test Set:")
print(classification_report(y_test, y_pred_ada_1))

# Accuracy on training set
y_train_pred = adaboost_1.predict(X_train)
train_acc = accuracy_score(y_train, y_train_pred)
print('Training Accuracy: %.2f%%' % (train_acc * 100))

# Accuracy on test set
test_acc = accuracy_score(y_test, y_pred_ada_1)
print('Test Accuracy: %.2f%%' % (test_acc * 100))

# Calculate final recall on the test set
test_recall_ada_1 = recall_score(y_test, y_pred_ada_1)
print('Final Model Recall on Test Set:  %.2f%%' % (test_recall_ada_1 * 100))


Classification Report on Test Set:
              precision    recall  f1-score   support

       False       0.91      0.99      0.95     10197
        True       0.70      0.17      0.28      1277

    accuracy                           0.90     11474
   macro avg       0.80      0.58      0.61     11474
weighted avg       0.88      0.90      0.87     11474

Training Accuracy: 90.03%
Test Accuracy: 89.97%
Final Model Recall on Test Set:  17.38%


In [66]:
# Get the report as a dictionary
report = classification_report(y_test, y_pred_ada_1, output_dict=True)

# Adding an entry to the model results list
all_model_results.append({
    'model': 'AdaBoost (n=50, lr=1)',
    'train_acc': accuracy_score(y_train, y_train_pred),
    'test_acc': report['accuracy'],
    'test_recall': recall_score(y_test, y_pred_ada_1),
    'precision_macro': report['macro avg']['precision'],
    'recall_macro': report['macro avg']['recall'],
    'f1_macro': report['macro avg']['f1-score'],
    'precision_weighted': report['weighted avg']['precision'],
    'recall_weighted': report['weighted avg']['recall'],
    'f1_weighted': report['weighted avg']['f1-score']
})

The AdaBoost model, configured with 50 estimators and a learning rate of 1, achieved a final recall of 17.38% on the test set. This really low score means that the model is failing to identify about 83% of actual customers with subscriptions--much lower than the final SVM model (70.48%). Despite this, the model showed high precision (70%) meaning that there's a strong chance that any customer predicted to subscribe actually does so. The high accuracy of 90% is misleading due to the imbalanced dataset, even though it is better than the SVM model (72.01%).

#### AdaBoost 2

A resampled dataset to balance the dataset and an optimized Adaboost model using a range of estimators (20-300 in increments of 25) and learning rates (0-1 in increments of 0.05) using 5-fold cross validation.

In [67]:
# Firstly, we will balance the dataset by undersampling the majority class
undersampler = RandomUnderSampler(random_state=1)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

print(f"Resample training set size: {X_train_resampled.shape[0]:,}")

Resample training set size: 5,962


In [68]:
# Now we'll set up our grid search
param_grid = {
    'n_estimators': list(range(20, 301, 25)),
    'learning_rate': [round(x, 2) for x in np.arange(0.1, 1.01, 0.05)],
}

# Creating our AdaBoost classifier
ada_boost_2 = AdaBoostClassifier(random_state=1)

grid_search_ada = GridSearchCV(
    estimator=ada_boost_2,
    param_grid=param_grid,
    scoring='recall',
    cv=5,
    n_jobs=-1 # With our wide range of hyperparameters, we're going to need to use all available cores to speed up the training
)

# Run the grid search on the resampled training data
grid_search_ada.fit(X_train_resampled, y_train_resampled)

GridSearchCV(cv=5, estimator=AdaBoostClassifier(random_state=1), n_jobs=-1,
             param_grid={'learning_rate': [np.float64(0.1), np.float64(0.15),
                                           np.float64(0.2), np.float64(0.25),
                                           np.float64(0.3), np.float64(0.35),
                                           np.float64(0.4), np.float64(0.45),
                                           np.float64(0.5), np.float64(0.55),
                                           np.float64(0.6), np.float64(0.65),
                                           np.float64(0.7), np.float64(0.75),
                                           np.float64(0.8), np.float64(0.85),
                                           np.float64(0.9), np.float64(0.95),
                                           np.float64(1.0)],
                         'n_estimators': [20, 45, 70, 95, 120, 145, 170, 195,
                                          220, 245, 270, 295]},
             scoring='recall')

In [69]:
print(f"Best Hyperparameters Found: {grid_search_ada.best_params_}")
best_ada = grid_search_ada.best_estimator_

# Display a more detailed classification report
y_pred_ada2 = best_ada.predict(X_test)
print("\nClassification Report on Test Set:")
print(classification_report(y_test, y_pred_ada2))

# Accuracy on (balanced) training set
y_train_pred = best_ada.predict(X_train_resampled)
train_acc = accuracy_score(y_train_resampled, y_train_pred)
print('Training Accuracy: %.2f%%' % (train_acc * 100))
print('Best Cross-Validation Recall Score on Training Set:  %.2f%%' % (grid_search_ada.best_score_ * 100))

# Accuracy on test set
test_acc = accuracy_score(y_test, y_pred_ada2)
print('Test Accuracy: %.2f%%' % (test_acc * 100))

# Calculate final recall on the test set
test_recall_ada2 = recall_score(y_test, y_pred_ada2)
print('Final Model Recall on Test Set:  %.2f%%' % (test_recall_ada2 * 100))

Best Hyperparameters Found: {'learning_rate': np.float64(0.85), 'n_estimators': 20}

Classification Report on Test Set:
              precision    recall  f1-score   support

       False       0.95      0.74      0.83     10197
        True       0.25      0.70      0.37      1277

    accuracy                           0.73     11474
   macro avg       0.60      0.72      0.60     11474
weighted avg       0.87      0.73      0.78     11474

Training Accuracy: 71.50%
Best Cross-Validation Recall Score on Training Set:  70.24%
Test Accuracy: 73.26%
Final Model Recall on Test Set:  69.77%


In [70]:
# Get the report as a dictionary
report = classification_report(y_test, y_pred_ada2, output_dict=True)

# Adding an entry to the model results list
all_model_results.append({
    'model': 'AdaBoost (GridSearch, Undersampled)',
    'train_acc': accuracy_score(y_train_resampled, y_train_pred),
    'test_acc': report['accuracy'],
    'test_recall': recall_score(y_test, y_pred_ada2),
    'precision_macro': report['macro avg']['precision'],
    'recall_macro': report['macro avg']['recall'],
    'f1_macro': report['macro avg']['f1-score'],
    'precision_weighted': report['weighted avg']['precision'],
    'recall_weighted': report['weighted avg']['recall'],
    'f1_weighted': report['weighted avg']['f1-score'],
    'best_cv_score': grid_search_ada.best_score_,
    'best_params': grid_search_ada.best_params_
})

From our grid search, our optimal AdaBoost model has a learning rate of 0.85 and 20 estimators. This method seems to be highly effective as we have a high recall indicating that we are able to correctly identify 69.77% of actual subscribers in the test set, only slightly worse than the final SVM at 70.48%. However, the precision is quite low at 25% meaning that many of our predictions are false positives--slightly more false positives than the 24% in the SVM. Despite this, the AdaBoost model still have a fairly good accuracy of 73.26%, which is 1% better than the SVM model at 72.01%.

### Summary Figure

Now that we've completed our last model, we'll display all of our results here in a figure for easy comparison:

In [77]:
results_df = pd.DataFrame(all_model_results)
results_df.set_index('model', inplace=True)
sorted_results_df = results_df.sort_values(by='recall_macro', ascending=False)

pct_cols = ['train_acc','test_acc','test_recall',
            'precision_macro','recall_macro','f1_macro',
            'precision_weighted','recall_weighted','f1_weighted',
            'best_cv_score']

fmt = results_df.copy()
for c in pct_cols:
    if c in fmt.columns:
        fmt[c] = (fmt[c]).round(4)

# sort by recall_macro as before and display
fmt = fmt.sort_values(by='recall_macro', ascending=False)
print("--- Model Comparison Table (Sorted by Macro Recall) ---")
fmt

--- Model Comparison Table (Sorted by Macro Recall) ---


,train_acc,test_acc,test_recall,precision_macro,recall_macro,f1_macro,precision_weighted,recall_weighted,f1_weighted,best_cv_score,best_params
model,,,,,,,,,,,
"AdaBoost (GridSearch, Undersampled)",0.7150,0.7326,0.6977,0.6003,0.7174,0.5990,0.8730,0.7326,0.7789,0.7024,"{'learning_rate': 0.85, 'n_estimators': 20}"
"SVM (GridSearch, Undersampled)",0.7115,0.7201,0.7048,0.5961,0.7134,0.5900,0.8722,0.7201,0.7695,0.7041,"{'C': 1, 'gamma': 0.001}"
Random Forest (n=25),0.9915,0.8932,0.2968,0.7267,0.6324,0.6619,0.8743,0.8932,0.8793,NaN,NaN
Random Forest (GridSearch),0.9915,0.8932,0.2968,0.7267,0.6324,0.6619,0.8743,0.8932,0.8793,0.2814,"{'max_depth': None, 'n_estimators': 25}"
SVM (GridSearch),0.9176,0.8973,0.2412,0.7537,0.6103,0.6438,0.8764,0.8973,0.8774,0.2452,"{'C': 10, 'gamma': 'scale'}"
SVM (rbf),0.9051,0.8982,0.2130,0.7671,0.5985,0.6314,0.8774,0.8982,0.8752,NaN,NaN
SVM (Linear),0.8983,0.8978,0.1926,0.7705,0.5894,0.6202,0.8766,0.8978,0.8726,NaN,NaN
Decision Tree (Depth 4),0.9006,0.8995,0.1770,0.7974,0.5835,0.6138,0.8816,0.8995,0.8720,NaN,NaN
"AdaBoost (n=50, lr=1)",0.9003,0.8997,0.1738,0.8018,0.5822,0.6122,0.8824,0.8997,0.8718,NaN,NaN


## Conclusion

> Make your conclusions from your analysis. Please be sure to address the business impact (it could be of any domain) of your solution.